In [1]:
%load_ext cython
import sys
sys.path.append('/home/bram/ANTS/')

In [12]:
%%cython -a
from cythonic.core.queen cimport queen
import numpy as np
cdef int n = 1000
cdef double dt = .1
cdef queen ant_queen = queen(n)
cdef str fun = 'linear'
cdef double[:] Q = np.array([1,1],dtype =np.float_)
ant_queen.deploy()

from time import time
cdef int N = 1000, i =0
cdef double tic, toc
tic = time()
for i in range(N):
    ant_queen.gradient_step(dt,fun,Q)
toc = time()
print(f"Avg {1000*(toc-tic)/n/N} msecs per ant")

Avg 0.0009363176822662354 msecs per ant


In [10]:
%%cython -a
from cythonic.core.ant cimport Ant
import numpy as np
from cythonic.plugins.positions cimport point
from time import time

# from libc.stdlib cimport malloc, free

# cdef float_type_t *list_of_arrays = \
#     <float_type_t *>malloc(no_of_arrays * sizeof(float_type_t*))

# for i in range(no_of_arrays):
#     list_of_arrays[i] = &(data_array[i].data)
cimport cython
@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
cdef class queen():
    cdef long n
    cdef list ants
    
    def __cinit__(self,long n):
        self.n = n
        self.ants = []
        
    cpdef void deploy(self, ):
        ant_dict = {'speed' : 10, 'gain' : 2, 'l' : 5, 'sens_offset' : 45,
          'limits' :np.array([10,10],dtype=np.float_),'q' : 1.5, 'return_factor' :1,
          'drop_fun' : 'exp_decay', 'drop_beta' : .5, 'rng_gamma':2}
        cdef int i
        cdef Ant a
        for i in range(self.n):
            a = Ant(id = i,**ant_dict)
            self.ants.append(a)
            
    cpdef void reverse(self):
        for ant in self.ants:
            ant.reverse()

    cpdef point gradient_step(self,double dt):
        "Gradient step wrapper"
        for ant in self.ants:
            ant.gradient_step(dt,observe_fun = 'linear',Q=np.array([1,1],dtype=np.float_))

    cpdef void observe_pheromone(self,fun, Q, fun_args = {}):
        "observe pheromone wrapper"
        # print(Q)eposit quantity time constant
        for i in range(self.n):
            self.ants[i].observe_pheromone(fun,Q[i],fun_args)

        
cdef queen Q = queen(10)
Q.deploy()
Q.gradient_step(.1)

In [9]:
from cythonic.ant_wrapper import pnt, sens
print(pnt(3.,5.))
from cythonic.ant_wrapper import pyAnt, rotate
import numpy as np


A = pyAnt(id =1, speed = 10, gain = 2, l = 5, sens_offset = 45,
          limits = np.array([10,10],dtype=np.float_),q = 1.5, return_factor = 1,
          drop_fun = 'exp_decay', drop_beta = .5, rng_gamma=2)

print(A.attributes())
print(A.foodbound   )

A.azimuth = 360
print(A.azimuth)

print(pnt(3.,2.))

# A.init_positions(np.array([1,1], dtype=np.float_))
print(A.pos)
print(A.chck_bnds())
print(A.return_positions())

print(sens('linear', 10))
print(A.return_observed('linear', [1.,2.]))
# print(A.return_drop_quantity())
A.gradient_step(.5,'linear', np.array([1,1], dtype=np.float_))

print(A.time)
# print(A.return_drop_quantity())

n = 1e5

t_observe = A.time_observe(n)
t_step = A.time_step(n)
t_sensors  = A.time_sensors(n)
t_full = A.time_iteration(n)
print("Observe took avg {} msec ".format(A.time_observe(n)/n))
print("Step took avg {} msec ".format(t_step/n))
print("Set sensors took avg {} msec".format(t_sensors/n))
print("Full ant iteration took avg {} msec".format(t_full/n))


{'x': 3.0, 'y': 5.0}
['_azimuth', '_drop_quantity', '_left', '_pos', '_right', 'azimuth', 'chck_bnds', 'drop_beta', 'drop_fun', 'foodbound', 'gain', 'gradient_step', 'id', 'l', 'left', 'limits', 'out_of_bounds', 'pos', 'q_observed', 'return_factor', 'return_observed', 'return_positions', 'right', 'rng', 'sens_offset', 'time', 'time_iteration', 'time_observe', 'time_sensors', 'time_step', 'v']
False
0.0
{'x': 3.0, 'y': 2.0}
(0.0, 0.0)
True
[array({'x': 0.0, 'y': 0.0}, dtype=object), array({'x': 0.0, 'y': 0.0}, dtype=object), array({'x': 0.0, 'y': 0.0}, dtype=object)]
10.0
[1. 2.]
0.5
Observe took avg 0.0006471514701843262 msec 
Step took avg 1.1854171752929687e-05 msec 
Set sensors took avg 3.9470195770263675e-05 msec
Full ant iteration took avg 0.0008214163780212402 msec
